In [1]:
import numpy as np
import qutip as qt
import qutip.qip.operations as op
from scipy.linalg import expm

from PulseSequence import PulseSequence
from Pulse import Pulse

### Thermal density matrix

In [2]:
pho_th = qt.Qobj(np.diag([1, 0.6, -0.6, -1]), dims=([[2, 2], [2, 2]]))
pho_th

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -0.6  0. ]
 [ 0.   0.   0.  -1. ]]

# OPERATORS

In [3]:
x = qt.sigmax()*0.5
y = qt.sigmay()*0.5
z = qt.sigmaz()*0.5
I = qt.qeye(2)

In [4]:
Ix = qt.tensor(x, I)
Iy = qt.tensor(y, I)
Iz = qt.tensor(z, I)

In [5]:
Sx = qt.tensor(I, x)
Sy = qt.tensor(I, y)
Sz = qt.tensor(I, z)

In [6]:
J = 215
tJ = 1/(2*J)
IzSz = qt.tensor(z, z)

In [7]:
Pulse("+2IzSz", IzSz, (2*np.pi*J)*(tJ) ).apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [8]:
# Delay for negative J coupling = 3*tJ; accurate upto global phase
Pulse("-2IzSz", IzSz, (2*np.pi*J)*(3*tJ) ).apply()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.70710678-0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.70710678+0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.70710678+0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.70710678-0.70710678j]]

We cant implement an $I_z/S_z$ pulse directly.  
Use $ R_z(\theta) = R_x(-\pi/2).R_y(\theta).R_x(\pi/2)$ 

In [9]:
def Iz_pulse(theta):
    return PulseSequence("Iz(theta)")\
        .add(Pulse("Ix(-pi/2)", Ix, -np.pi/2))\
        .add(Pulse(f"Iy({theta/np.pi}*pi)", Iy, theta))\
        .add(Pulse("Ix(pi/2)", Ix, np.pi/2))

In [10]:
Iz_pulse(np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

In [11]:
(-1j*np.pi/2*Iz).expm()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

Verified that $R_x(-\pi/2).R_y(\pi/2).R_x(\pi/2) = R_z(\pi/2)$ 

In [12]:
Iz_pulse(-np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [13]:
(+1j*np.pi/2*Iz).expm()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

Verified that $R_x(-\pi/2).R_y(-\pi/2).R_x(\pi/2) = R_z(-\pi/2)$ 

In [14]:
def Sz_pulse(theta):
    return PulseSequence("Sz(theta)")\
        .add(Pulse("Sx(-pi/2)", Sx, -np.pi/2))\
        .add(Pulse(f"Sy({theta/np.pi}*pi)", Sy, theta))\
        .add(Pulse("Sx(pi/2)", Sx, np.pi/2))

In [15]:
Sz_pulse(np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678-0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678-0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]

In [16]:
Sz_pulse(np.pi/2).print_sequence()

Sz(theta) Pulse Sequence: Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2)

In [17]:
Sz_pulse(-np.pi/2).compile().get_operator()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.70710678-0.70710678j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.70710678+0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678-0.70710678j]]

In [18]:
Sz_pulse(-np.pi/2).print_sequence()

Sz(theta) Pulse Sequence: Sx(-pi/2) -> Sy(-0.5*pi) -> Sx(pi/2)

# Sample Gates

### Two Qubit Controlled Z

In [19]:
cz_p = PulseSequence("CZ")\
    .add_seq(Iz_pulse(np.pi/2))\
    .add_seq(Sz_pulse(np.pi/2))\
    .add(Pulse("-2IzSz(aka delay(3/2J))", IzSz, 3*np.pi))

In [20]:
print(f"CZ multiplied by phase factor:")
np.exp(1j*np.pi/4)*cz_p.compile().get_operator()

CZ multiplied by phase factor:


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]]

In [21]:
cz_p.print_sequence()

CZ Pulse Sequence: Ix(-pi/2) -> Iy(0.5*pi) -> Ix(pi/2) -> Sx(-pi/2) -> Sy(0.5*pi) -> Sx(pi/2) -> -2IzSz(aka delay(3/2J))

### Two Qubit Near-CNOT

In [22]:
ncx = PulseSequence("Near CNOT")\
    .add(Pulse("Sx(pi/2)", Sx, np.pi/2))\
    .add(Pulse("2IzSz(aka delay(1/2J))", IzSz, np.pi))\
    .add(Pulse("Sy(-pi/2)", Sy, -np.pi/2))

In [23]:
print(f"Near CX multiplied by phase factor: ")
np.exp(-1j*np.pi/4)*ncx.compile().get_operator()

Near CX multiplied by phase factor: 


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.-1.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]]

In [24]:
ncx.print_sequence()

Near CNOT Pulse Sequence: Sx(pi/2) -> 2IzSz(aka delay(1/2J)) -> Sy(-pi/2)

In [25]:
ncx_rho_th = ncx.compile().evolve_pho(pho_th)
ncx_rho_th.rho

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   0.6  0.   0. ]
 [ 0.   0.  -1.   0. ]
 [ 0.   0.   0.  -0.6]]

In [26]:
ncx_rho_th.h_spectrum()
ncx_rho_th.c_spectrum()

H spectrum
Peak1 at 4.7364 ppm with integral=(2+0j)
Peak2 at 8.1836 ppm with integral=(1.2+0j)
C spectrum
Peak1 @ 66.4751 ppm with integral=0j
Peak2 @ 80.1869 ppm with integral=(-0+0j)
